In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf

from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-28 23:31:48.127008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
sampler = RNNSampler(BASE_TOKENS, 16, 1)

2023-12-28 23:31:49.588214: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
root - DEBUG - 28-Dec-23 23:31:49 - Raw output, object=RNNSampler, func=call, probs=<Tensor shape: (32, 10)
[[0.11, 0.12, ..., 0.03, 0.09]
 [0.11, 0.12, ..., 0.03, 0.09]
 ...
 [0.11, 0.12, ..., 0.03, 0.09]
 [0.11, 0.12, ..., 0.03, 0.09]]>
root - DEBUG - 28-Dec-23 23:31:49 - object=RNNSampler, raw_probs=<Tensor shape: (32, 10)
[[0.11, 0.12, ..., 0.03, 0.09]
 [0.11, 0.12, ..., 0.03, 0.09]
 ...
 [0.11, 0.12, ..., 0.03, 0.09]
 [0.11, 0.12, ..., 0.03, 0.09]]>
root - DEBUG - 28-Dec-23 23:31:49 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (32, 10)
[[0.11, 0.12, ..., 0.00, 0.00]
 [0.11, 0.12, ..., 0.00, 0.00]
 ...
 [0.11, 0.12, ..., 0.00, 0.00]
 [0.11, 0.12, ..., 0.00, 0.00]]>
root - DEBUG - 28-Dec-23 23:31:49 - After normalization, object=RNNSampler, func=call, probs

In [3]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [4]:
sampler.sample(3)

root - DEBUG - 28-Dec-23 23:31:56 - Raw output, object=RNNSampler, func=call, probs=<Tensor shape: (3, 10)
[[0.11, 0.12, ..., 0.03, 0.09]
 [0.11, 0.12, ..., 0.03, 0.09]
 [0.11, 0.12, ..., 0.03, 0.09]]>
root - DEBUG - 28-Dec-23 23:31:56 - object=RNNSampler, raw_probs=<Tensor shape: (3, 10)
[[0.11, 0.12, ..., 0.03, 0.09]
 [0.11, 0.12, ..., 0.03, 0.09]
 [0.11, 0.12, ..., 0.03, 0.09]]>
root - DEBUG - 28-Dec-23 23:31:56 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (3, 10)
[[0.11, 0.12, ..., 0.00, 0.00]
 [0.11, 0.12, ..., 0.00, 0.00]
 [0.11, 0.12, ..., 0.00, 0.00]]>
root - DEBUG - 28-Dec-23 23:31:56 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (3, 10)
[[0.12, 0.14, ..., 0.00, 0.00]
 [0.12, 0.14, ..., 0.00, 0.00]
 [0.12, 0.14, ..., 0.00, 0.00]]>
root - DEBUG - 28-Dec-23 23:31:56 - Raw output, object=RNNSampler, func=call, probs=<Tensor shape: (3, 10)
[[0.09, 0.11, ..., 0.05, 0.09]
 [0.08, 0.11, ..., 0.04, 0.11]
 [0.09, 0.11, ..., 0.04, 0

(<tf.Tensor: shape=(3, 15), dtype=int32, numpy=
 array([[2, 4, 4, 5, 5, 1, 8, 1, 3, 2, 8, 9, 9, 9, 9],
        [1, 0, 4, 5, 6, 0, 2, 5, 9, 4, 9, 8, 9, 9, 9],
        [0, 5, 0, 9, 1, 2, 3, 7, 1, 8, 9, 9, 8, 9, 9]], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([15, 14, 15], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([24.322533, 23.709707, 22.44196 ], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-26.018202, -25.179129, -25.573078], dtype=float32)>)

In [ ]:
sequences, lengths, entropies, log_probs = sampler.sample(3)

In [ ]:
ensemble = ExpressionEnsemble(sequences, lengths)

In [ ]:
[expr.tree for expr in ensemble]

In [ ]:
regressor = SymbolicRegressor(sampler, n_samples=16, loss_func=SymbolicLoss())

In [ ]:
outputs = regressor({"X": X, "y": y})

In [ ]:
regressor.fit(X, y)

# 

In [6]:
sampler = RNNSampler(BASE_TOKENS, 16, 1)
loss_func = SymbolicLoss()

root - DEBUG - 28-Dec-23 23:32:20 - Raw output, object=RNNSampler, func=call, probs=<Tensor shape: (32, 10)
[[0.18, 0.06, ..., 0.04, 0.08]
 [0.18, 0.06, ..., 0.04, 0.08]
 ...
 [0.18, 0.06, ..., 0.04, 0.08]
 [0.18, 0.06, ..., 0.04, 0.08]]>
root - DEBUG - 28-Dec-23 23:32:20 - object=RNNSampler, raw_probs=<Tensor shape: (32, 10)
[[0.18, 0.06, ..., 0.04, 0.08]
 [0.18, 0.06, ..., 0.04, 0.08]
 ...
 [0.18, 0.06, ..., 0.04, 0.08]
 [0.18, 0.06, ..., 0.04, 0.08]]>
root - DEBUG - 28-Dec-23 23:32:20 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (32, 10)
[[0.18, 0.06, ..., 0.00, 0.00]
 [0.18, 0.06, ..., 0.00, 0.00]
 ...
 [0.18, 0.06, ..., 0.00, 0.00]
 [0.18, 0.06, ..., 0.00, 0.00]]>
root - DEBUG - 28-Dec-23 23:32:20 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (32, 10)
[[0.20, 0.06, ..., 0.00, 0.00]
 [0.20, 0.06, ..., 0.00, 0.00]
 ...
 [0.20, 0.06, ..., 0.00, 0.00]
 [0.20, 0.06, ..., 0.00, 0.00]]>


In [12]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())


X = pandas_to_tensor(X)
y = tf.convert_to_tensor(y, dtype=tf.float32)
constants = tf.Variable([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [13]:
tree

exp(const / const / const + const) / var_x + const * const

In [14]:
with tf.GradientTape() as tape:
    sequences, lengths, entropies, log_probs = sampler.sample(10)
    y_pred = []
    for sequence, length in zip(sequences, lengths):
        tree = Node.from_sequence(sequence[:length], BASE_TOKENS)
        y_pred.append(tree.tf_eval(X, constants=constants))
    y_pred = tf.stack(y_pred)
    loss = loss_func(y, y_pred, entropies, log_probs)

grads = tape.gradient(loss, sampler.variables)

root - DEBUG - 28-Dec-23 23:33:42 - Raw output, object=RNNSampler, func=call, probs=<Tensor shape: (10, 10)
[[0.18, 0.06, ..., 0.04, 0.08]
 [0.18, 0.06, ..., 0.04, 0.08]
 ...
 [0.18, 0.06, ..., 0.04, 0.08]
 [0.18, 0.06, ..., 0.04, 0.08]]>
root - DEBUG - 28-Dec-23 23:33:42 - object=RNNSampler, raw_probs=<Tensor shape: (10, 10)
[[0.18, 0.06, ..., 0.04, 0.08]
 [0.18, 0.06, ..., 0.04, 0.08]
 ...
 [0.18, 0.06, ..., 0.04, 0.08]
 [0.18, 0.06, ..., 0.04, 0.08]]>
root - DEBUG - 28-Dec-23 23:33:42 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (10, 10)
[[0.18, 0.06, ..., 0.00, 0.00]
 [0.18, 0.06, ..., 0.00, 0.00]
 ...
 [0.18, 0.06, ..., 0.00, 0.00]
 [0.18, 0.06, ..., 0.00, 0.00]]>
root - DEBUG - 28-Dec-23 23:33:42 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (10, 10)
[[0.20, 0.06, ..., 0.00, 0.00]
 [0.20, 0.06, ..., 0.00, 0.00]
 ...
 [0.20, 0.06, ..., 0.00, 0.00]
 [0.20, 0.06, ..., 0.00, 0.00]]>
root - DEBUG - 28-Dec-23 23:33:42 - Raw output,

['Variable:0', 'Variable:0', 'rnn_sampler_1/simple_rnn_1/simple_rnn_cell_1/kernel:0', 'rnn_sampler_1/simple_rnn_1/simple_rnn_cell_1/recurrent_kernel:0', 'rnn_sampler_1/simple_rnn_1/simple_rnn_cell_1/bias:0', 'rnn_sampler_1/dense_1/kernel:0', 'rnn_sampler_1/dense_1/bias:0', 'Variable:0']


In [15]:
grads

[<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
 array([[-4.64325920e-02, -1.34979010e-01,  1.04090422e-01,
          1.23985663e-01,  8.25244263e-02, -1.05006574e-02,
          1.45106748e-01,  1.11135747e-03,  4.10630107e-02,
         -2.13235795e-01,  1.61248744e-01,  2.13754326e-02,
          2.20935494e-02,  1.17306337e-01, -1.73590213e-01,
          3.04358695e-02],
        [ 1.08299339e-02,  1.06238239e-02,  1.36922958e-04,
          9.28509608e-03,  6.86116284e-04,  1.72782727e-02,
          1.87520031e-02, -4.34505194e-03,  1.25176376e-02,
         -4.03443025e-03,  5.11503872e-03,  1.42675582e-02,
          6.33129734e-04, -5.03229117e-03, -1.45407449e-02,
          5.81811601e-03],
        [ 4.46275622e-02,  4.37782295e-02,  5.64226648e-04,
          3.82616520e-02,  2.82732048e-03,  7.11996183e-02,
          7.72725120e-02, -1.79049168e-02,  5.15821837e-02,
         -1.66249182e-02,  2.10778471e-02,  5.87931834e-02,
          2.60897563e-03, -2.07368638e-02, -5.99189214e

# TODO constraint avoid x / x

In [ ]:
from tensorflow.keras.losses import MSE
from scipy.optimize import basinhopping, minimize

In [ ]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [ ]:
expression = Expression(tree)

In [ ]:
results = {}
for T in np.logspace(0, 2, 10):
    res = basinhopping(
        lambda constants: MSE(y, expression.eval(X, constants)),
        expression.constants,
        T=T,
    )
    results[T] = res

In [ ]:
[res.x for result in results]

In [ ]:
res = expression.fit(X, y)

In [ ]:
expression.constants = res.x

In [ ]:
go.Figure(
    [go.Scatter(y=expression.eval(X, res.x)), go.Scatter(y=expression.eval(X, [2, 1]))]
)

In [ ]:
model = EvalModel(tree)
model.compile(optimizer=RMSprop(learning_rate=0.05), loss="mse")
model.fit(
    X,
    y,
    batch_size=X.shape[0],
    epochs=1000,
    callbacks=[
        EarlyStopping(monitor="loss", patience=10, start_from_epoch=10, min_delta=0)
    ],
)
model.constants

In [ ]:
def step(tree, X, y, constants, optimizer: Optimizer):
    with tf.GradientTape() as tape:
        y_pred = walk(tree, tf_eval, X=X, constants={"iter": 0, "value": constants})
        loss = MSE(y, y_pred)
    grads = tape.gradient(loss, constants)
    optimizer.apply_gradients([(grads, constants)])
    return grads, loss


def optimize(
    tree, X: pd.DataFrame, y: pd.Series, constants: tf.Variable, max_steps: int = 100
):
    # optimizer = SGD(learning_rate=0.1)
    optimizer = RMSprop(learning_rate=0.1)

    if isinstance(X, pd.DataFrame):
        X = format_as_dict_of_tensor(X)

    if isinstance(y, pd.Series):
        y = tf.convert_to_tensor(y)

    for n in range(max_steps):
        grads, loss = step(tree, X, y, constants, optimizer)
    return constants


def optimize_constants(tree, X: pd.DataFrame, y: pd.Series, max_steps: int = 100):
    num_constants = walk(tree, count_constant)
    constants = tf.Variable(np.random.randn(num_constants), dtype=tf.float32)
    return optimize(tree, X, y, constants, max_steps)

In [ ]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [ ]:
model.fit(X, y)

In [ ]:
xv, yv = np.meshgrid(np.linspace(-4, 4), np.linspace(-4, 4))

In [ ]:
losses = np.empty_like(xv)
grads = np.empty((50, 50, 2))
for i in range(50):
    for j in range(50):
        constants = tf.Variable([xv[i, j], yv[i, j]])
        with tf.GradientTape() as tape:
            y_pred = walk(tree, tf_eval, X=X, constants={"iter": 0, "value": constants})
            loss = MSE(y, y_pred)
        losses[i, j] = loss.numpy()
        grad = tape.gradient(loss, constants)
        grads[i, j] = grad.numpy()

In [ ]:
go.Figure(
    go.Contour(x=np.linspace(-4, 4), y=np.linspace(-4, 4), z=losses, colorscale="RdBu"),
)

In [ ]:
ff.create_quiver(xv, yv, grads[:, :, 0], grads[:, :, 1], scale=0.05, arrow_scale=0.25)

In [ ]:
losses

In [ ]:
grad

In [ ]:
constants = optimize_constants(tree, X, y)

In [ ]:
constants

In [ ]:
go.Figure(
    [
        go.Scatter(
            x=X.squeeze(),
            y=walk(
                tree, tf_eval, X=X, constants={"iter": 0, "value": constants}
            ).numpy(),
        ),
        go.Scatter(x=X.squeeze(), y=y),
    ]
)